# Samarium Fluorescence

This notebook computes the expected fluorescence signal and Rabi frequency for the Samarium experiment given currently measured/assumed parameters. Parameters/formulae are generally chosen to be consistent with notation in the textbook 'Lasers' by Milonni and Eberly. Most expressions are from chapter 7.

In [1]:
#Headers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Functions
def Lorentz(omega,omega_0,Gamma):
    return 1/(np.pi * (Gamma/2)) * 1 / (1 + ((omega - omega_0)/(Gamma/2))**2)

In [3]:
#Constants
c = 2.998E8 #Speed of light in m/s
h = 6.626E-34 #Planck's constant in J/Hz
NA = 6.022E23 #Avagadro's number
e = 1.602E-19 #Electron charge in C

cm = 1E-2 #m
um = 1E-6 #m
nm = 1E-9 #m
ns = 1E-9 #s
fs = 1E-15 #s
uA = 1E-6 #A
pA = 1E-12 #A
mW = 1E-3 #W
MHz = 1E6 #Hz
GHz = 1E9 #Hz
THz = 1E12 #Hz

Measured optical properties of the beam and atom. Note that the atomic cross-section is not actually directly measured, because we do not take into account the linewidth of the laser: larger laser linewidths would result in smaller fractions of atoms transferred to the excited state, since less of the laser is resonant with the atom, leading to a smaller effective cross-section. It is better to just assume we measure a total excitation rate, and infer the cross-section from this, given knowledge of the laser linewidth. 

Even with an unfocussed beam, the inferred stimulated absorption rate is comparable to the excited state linewidth.

In [32]:
#Optical properties
nu = 438.035 * THz #Centre of the inhomogeneous beam profile in Hz
lam = c/nu / nm #Clock transition in nm (vacuum wavelength)
P = 190 #Peak beam power after TA in mW
w = 0.079 #Beam waist through crystal in cm (to be checked)
I = P/np.pi/w**2 #Average intensity in mW/cm^2
Phi = (I * mW)/(h * nu) #Average photon flux in photons/cm^2/s
Gamma_L = 2 * np.pi * 300 #Measured Laser linewidth in 1/s
Gamma_Ex = 2 * np.pi * 13 #Measured excited state linewidth in 1/s
sigma = 1.3E-18 #Assumed atomic cross-section in cm^2 - We don't actually measure this! We just measure a stimulated absorption rate.
R12 = 1.3E-18 * Phi #Measured stimulated absorption rate in 1/s. This is what we actually fit in a saturation curve.
Eff = 0.5 * 0.01 #A guess for expected photodiode efficiency * collection efficiency.


print(f'Frequency is {nu/THz:.3f} THz.')
print(f'Clock transition vacuum wavelength is {lam:.3f} nm.')
print(f'Intensity is {I:.3f} mW/cm^2.')
print(f'Photon flux is {Phi:.3e} photons/cm^2/s.')
print(f'Measured stimulated absorption rate is {R12:.3f}/s, or {R12/Gamma_Ex:.3f} excited state linewidths.')

Frequency is 438.035 THz.
Clock transition vacuum wavelength is 684.420 nm.
Intensity is 9690.575 mW/cm^2.
Photon flux is 3.339e+19 photons/cm^2/s.
Measured stimulated absorption rate is 43.404/s, or 0.531 excited state linewidths.


Properties of the crystal. We just treat the I=7/2 isotopes for now, assuming these are the ones that are inhomogeneously broadened. There must be an assumed value for the amount of homogeneous broadening of the transition: this amount should be greater than or equal to the excited state linewidth. It seems to need to be greater than the linewidth to be consistent with measurements.

In [33]:
#Crystal properties
L = 1 #Length of the fluorescing region in cm
rho_SrF2 = 4.24 #Density of SrF2 in g/cm^3 
prop = 0.0001 #Sm fraction by weight
m_Sm = np.array([143.912,146.915,147.915,148.917,149.917,151.912,153.922]) #Masses of Sm isotopes in g/mol
mole_fraction = np.array([0.031,0.150,0.112,0.138,0.074,0.268,0.228]) #Fraction of each isotope in natural Sm
m_Sm_av = np.sum(m_Sm * mole_fraction) #Averge mass of Sm in g/mol
n_Sm = rho_SrF2 * prop / m_Sm_av * NA #Number density of Sm in the crystal in atoms/cm^3
N_Inhom = (mole_fraction[1] + mole_fraction[3]) * n_Sm * L * np.pi * w**2 #Number of atoms with I = 7/2 in one beam volume
Gamma_In = 2 * np.pi * 30 * GHz #Rough width of inhomogeneously broadening profile in 1/s
Gamma_H = 1000 * Gamma_Ex #Assumed homogeneous broadening - this must be at least equal to the excited state linewidth

print(f'Average mass of Sm is {m_Sm_av:.3f} g/mol.')
print(f'Sm number density is {n_Sm:.3e} atoms/cm^3.')
print(f'Number of I=7/2 isotopes in one beam volume is {N_Inhom:.3e} atoms.')

Average mass of Sm is 150.516 g/mol.
Sm number density is 1.696e+18 atoms/cm^3.
Number of I=7/2 isotopes in one beam volume is 9.579e+15 atoms.


Here we calculate expected fluorescence and Rabi frequency. Consider a laser with total photon flux $\Phi_0$ and linewidth $\Gamma_L$ (FWHM) centred at a frequency $\omega_{c}$. Its photon flux per unit angular frequency $\omega$ could be modelled as a Lorentzian given by
$$\Phi(\omega - \omega_c; \Gamma_{L}) = \frac{\Phi_{0}}{\pi}\frac{\Gamma_{L}/2}{\left(\omega - \omega_{c}\right)^{2} + \left(\Gamma_{L}/2\right)^{2}} \equiv \Phi_{0} L(\omega - \omega_{c}; \Gamma_{L})$$
where $L(\omega - \omega_{c}; \Gamma_{L})$ is shorthand for a normalized Lorentzian function. 

Similarly, an individual atom with resonance frequency $\omega_0$ is broadened by its homogeneous linewidth $\Gamma_{H}$. Usually, one quotes a resonant cross-section, which for a two-level atom comes out to $\sigma_p = \lambda^2/2\pi$. However, for radiation incident off-resonance, the cross-section comes out to (check factors of 2)
$$\sigma(\omega - \omega_0; \Gamma_{H}) =  \sigma_p \frac{\pi \Gamma_{H}}{2} L(\omega - \omega_{0}; \Gamma_{H}) $$
Note that this quantity still has units of area.

Consider the laser incident on the atom. To find the total stimulated absorption rate, we should integrate over the frequency distribution of the intensity and multiply by the cross-section. Thus, we get
$$R_{12}(\omega_0) = \int \Phi(\omega - \omega_c; \Gamma_{L}) \sigma(\omega - \omega_0; \Gamma_{H}) d\omega $$
Define $\Delta = \omega - \omega_0$ as the detuning from atomic resonance, and $\Delta_{L} = \omega_c - \omega_0$ as the laser detuning from resonance. Then, this can be recast as
$$R_{12}(\omega_0) = \int \Phi(\Delta - \Delta_{L}; \Gamma_{L}) \sigma(\Delta; \Gamma_{H}) d\Delta$$
which is a convolution between the laser lineshape and the atomic lineshape. 

In the case of the 3-level Samarium system, assuming that ${}^7F_{1}$ decays much faster than ${}^5D_{0}$, the equation for the likelihood to find the atom resonant at $\omega_0$ in the excited state is given by
$$P_{Ex}(\omega_0) = \frac{R_{12}(\omega_0) / \Gamma_{Ex}}{1 + 2 R_{12}(\omega_0) / \Gamma_{Ex}} $$

Of course, in the SrF2 crystal, atoms have all sorts of resonant frequencies $\omega_0$ due to inhomogeneous broadening. Given the observed fluorescence distribtuion, we can approximate this distribution of atoms as a Lorentzian centred at the frequency of peak fluorescence $\omega_{In}$ with FWHM $\Gamma_{In} = 2\pi\times30$ GHz as
$$ n(\omega_0) = (N_{147} + N_{149}) L(\omega_{0} - \omega_{In}; \Gamma_{In}) $$.

Therefore, we can get the total fluorescence rate $\Gamma_{Sc}$ by integrating over all frequencies and weighting by the frequency of emitted fluorescent photons.
$$ \Gamma_{Sc} = \Gamma_{Ex} \int n(\omega_0) P_{Ex}(\omega_0) d\omega_0 $$

Finally, note that another expression for $P_{Ex}$ in terms of the Rabi frequency $\Omega_0$ for a laser detuned from atomic resonance is
$$P_{Ex}(\Delta) = \frac{\Omega_0^2 \Gamma_{H}/\Gamma_{Ex}}{2\Delta^{2} + 2\Gamma_{H}^2 + 2\Omega_0^2\Gamma_{H}/\Gamma_{Ex}} $$
Comparison between the expressions suggests
$$\Omega_0 = \sqrt{\Gamma_H R_{12}}$$
although, I should think about this more since this is for a monochromatic beam (i.e. $\Gamma_{L} << \Gamma_{H}$).

In [34]:
#Modelling the fluorescence/Rabi frequency
larger_line = np.max((Gamma_L,Gamma_H)) #Which is the limiting linewidth
d_omega = larger_line / 100001 #Set the angular frequency spacing
omega = np.arange(-3*Gamma_L/2, 3*Gamma_L/2, d_omega) #Relevant frequencies from laser beam in 1/s
omega_0 = np.arange(-3*Gamma_H/2, 3*Gamma_H/2, d_omega) #Relevant frequencies for the atoms in 1/s. 
omega_tot = np.linspace(-3*Gamma_L/2 - 3*Gamma_H/2,3*Gamma_L/2 + 3*Gamma_H/2,num=(len(omega) + len(omega_0) - 1))

Phi_omega = Phi * Lorentz(omega,0.0,Gamma_L) #Photon flux per unit frequency in photons/cm^s/s/(1/s)
sigma_omega_0 = sigma * (np.pi * Gamma_H / 2.0) * Lorentz(omega_0,0.0,Gamma_H) #Frequency-dependent cross-section in cm^2.

R_omega_0 = np.convolve(Phi_omega, sigma_omega_0, 'full') * d_omega #Stimulated absorption rate in photons/s/(1/s)

N_Inhom_omega_0 = N_Inhom * Lorentz(omega_tot,0.0,Gamma_In) #Number of  atoms per frequency bin in atoms/(1/s) 
Frac_omega_0 = R_omega_0 / (Gamma_Ex + 2 * R_omega_0) #Fraction of atoms excited in each frequency bin

Omega_0 = np.sqrt(np.max(R_omega_0) * Gamma_H) #The max Rabi frequency (angular) in rad/s
Gamma_Sc = Gamma_Ex * np.sum(N_Inhom_omega_0 * d_omega * Frac_omega_0) #Integrate over the inhomogeneous lineshape to get scattering rate in 1/s

print(f'Fraction of atoms available in three laser linewidths is {np.sum(N_Inhom_omega_0 * d_omega)/N_Inhom:.3e}.')
print(f'Peak fraction excited is {np.max(Frac_omega_0):.3f}.')
print(f'Calculated peak stimulated absorption rate is {np.max(R_omega_0)/Gamma_Ex:.3f} excited state linewidths, compared to {R12/Gamma_Ex:.3f} measured.')
print(f'Peak Rabi frequency is {Omega_0/2/np.pi:.3f} Hz.')
print(f'Fluorescence rate is {Gamma_Sc:.3e} /s.')
print(f'Expected current is {Gamma_Sc * Eff * e / pA:.3e} pA.')

Fraction of atoms available in three laser linewidths is 8.467e-07.
Peak fraction excited is 0.229.
Calculated peak stimulated absorption rate is 0.422 excited state linewidths, compared to 0.531 measured.
Peak Rabi frequency is 267.130 Hz.
Fluorescence rate is 7.694e+10 /s.
Expected current is 6.163e+01 pA.
